In [1]:
# ---- load globals ----
source("R/globals.R", local = TRUE)

source("R/data_loading.R", local = TRUE)


source("R/module_rotate.R", local = TRUE)

library(plotly)       
library(base64enc)    
source("R/module_select.R") 

source("R/download.R", local = TRUE)


source("R/plot.R", local = TRUE)

source("R/distance.R", local = TRUE)

source("R/gam_analysis.R", local = TRUE)

source("R/GO_utils.R", local = TRUE)

source("R/traj_utils.R", local = TRUE)


Attaching package: ‘jsonlite’


The following object is masked from ‘package:shiny’:

    validate



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout




In [2]:
ui <- fluidPage(
  titlePanel("Visium HE overlay + gene expression (Lv0)"),
  sidebarLayout(
    sidebarPanel(
      # ---- Data upload ----
      fileInput("matrix_file",   "Upload matrix.mtx(.gz)"),
      fileInput("barcodes_file", "Upload barcodes.tsv(.gz)"),
      fileInput("features_file", "Upload features.tsv(.gz)"),
      fileInput("positions_file","Upload tissue_positions_list.csv"),
      fileInput("scalef_file",   "Upload scalefactors_json.json"),
      fileInput("image_file",    "Upload HE image (png/jpg)"),

      # ---- Gene / appearance settings ----
      selectInput("gene", "Select gene:", choices = character(0), selected = NULL),
      checkboxInput("log1p", "log1p transform", value = TRUE),
      sliderInput("ptsize", "Point size", min = 1, max = 6, value = 3, step = 0.5),
      sliderInput("alpha", "Point transparency", min = 0.1, max = 1, value = 0.9, step = 0.1),
      helpText("Please upload the 10x matrix trio (matrix/barcodes/features) first before selecting genes."),
      tags$hr(),
      rotateImageUI("rot")
    ),

    mainPanel(
      # ================= Main plot & selection =================
      radioButtons(
        "sel_mode", "Selection tool",
        choices = c("Lasso" = "lasso", "Rectangle" = "select"),
        inline = TRUE, selected = "lasso"
      ),
      plotly::plotlyOutput("he_plotly", height = "750px", width = "100%"),

      # ================= Structure annotation =================
      hr(), h4("Structure annotation (biological structure)"),
      fluidRow(
          column(4,
            selectizeInput(
              "bio_label", "Biological structure",
              choices = c("tumor","tls","stroma"),
              selected = "tumor",
              options = list(create = TRUE, persist = TRUE)   # 可以键入新增
            )
          ),
        column(4, actionButton("add_annotation", "Annotate current selection as this structure")),
        column(4, actionButton("clear_annotations", "Clear all structure annotations"))
      ),
      tableOutput("anno_summary"),

      # ================= Export & tools =================
      hr(), h4("Export & tools"),
      fluidRow(
        column(3, actionButton("clear_sel", "Clear selection")),
        column(3, downloadButton("dl_sel", "Download selected CSV")),
        column(3, verbatimTextOutput("sel_count")),
        column(3, downloadButton("dl_multi_distance", "Download all structure distances"))
      ),
      fluidRow(
        column(4, downloadButton("dl_gam_results", "Download GAM fitting results"))
      ),

      # ================= Expression ~ distance trajectory (LOESS) =================
      hr(), h3("Expression trajectory along distance (with LOESS smoothing)"),
      fileInput("dist_csv", "Upload distance file (with barcode and dist_* columns; optional expr column)", accept = ".csv"),
      fluidRow(
        column(4, uiOutput("traj_dist_col_ui")),     # Dynamic: distance column selection
        column(3, numericInput("traj_bins",  "Number of bins", 50, min = 10, step = 10)),
        column(3, sliderInput("traj_span",   "LOESS span", min = 0.1, max = 1, value = 0.4, step = 0.05)),
        column(2, checkboxInput("traj_show_points", "Show scatter points", TRUE))
      ),
      plotOutput("traj_plot", height = "420px"),
      fluidRow(
        column(6, downloadButton("dl_traj_plot",  "Download trajectory plot (PNG)")),
        column(6, downloadButton("dl_traj_table", "Download binning statistics (CSV)"))
      ),
      tableOutput("traj_table"),

      # ================= GAM → GO enrichment =================
      hr(), h3("GAM results → Top genes → GO enrichment"),
      fileInput("gam_csv", "Upload gam_results_*.csv (must contain gene and p_dist_* columns)", accept = ".csv"),
      fluidRow(
        column(4, uiOutput("dist_col_ui")),  # Dynamically fetch p_dist_* columns
        column(3, numericInput("topn_go", "Top N genes", 20, min = 5, step = 5)),
        column(3, selectInput("go_org", "Species", c(Human = "human", Mouse = "mouse"), "human")),
        column(2, actionButton("run_go", "Run GO enrichment", class = "btn-primary"))
      ),
      br(),
      h4("Top genes (sorted by selected p column ascending)"),
      tableOutput("tbl_top_genes"),
      br(),
      h4("GO enrichment results (BP)"),
      tableOutput("tbl_go"),
      plotOutput("plot_go", height = "480px"),
      downloadButton("dl_go_table", "Download GO table"),

      hr(),
      verbatimTextOutput("dbg")
    )
  )
)




In [3]:
server <- function(input, output, session) {
  rv <- reactiveValues(
    counts=NULL, gene_names=NULL, barcodes=NULL,
    pos=NULL, hires_scale=NULL, img=NULL, img_w=NULL, img_h=NULL, img_path=NULL
  )

  # ---- Load counts/features/barcodes ----
  observeEvent(list(input$matrix_file, input$barcodes_file, input$features_file), {
    req(input$matrix_file, input$barcodes_file, input$features_file)
    paths <- list(input$matrix_file$datapath, input$barcodes_file$datapath, input$features_file$datapath)
    message("Loading counts ...")
    res <- load_counts(paths[[1]], paths[[2]], paths[[3]])
    rv$counts     <- res$counts
    rv$gene_names <- res$gene_names
    rv$barcodes   <- res$barcodes
    updateSelectizeInput(
        session, "gene",
        choices  = rv$gene_names,  # all genes handled on backend
        selected = character(0),   # don’t select by default
        server   = TRUE            # enable server-side search (performance critical)
      )
  })

  # ---- Load positions ----
  observeEvent(input$positions_file, {
    req(input$positions_file)
    message("Loading positions ...")
    pos <- load_positions(input$positions_file$datapath)
    if (!is.null(rv$counts)) {
      pos <- subset(pos, in_tissue == 1 & barcode %in% colnames(rv$counts))
    } else {
      pos <- subset(pos, in_tissue == 1)
    }
    rv$pos <- pos
  })

  # ---- Load scalefactor ----
  observeEvent(input$scalef_file, {
    req(input$scalef_file)
    message("Loading scalefactor ...")
    rv$hires_scale <- load_scalef(input$scalef_file$datapath)
    if (!is.null(rv$pos)) rv$pos <- to_hires_coords(rv$pos, rv$hires_scale)
  })

  # ---- Load image ----
  observeEvent(input$image_file, {
    req(input$image_file)
    message("Loading image ...")
    im <- load_image(input$image_file$datapath)
    rv$img   <- im$img      # raster
    rv$img_w <- im$img_w
    rv$img_h <- im$img_h
    rv$img_path <- input$image_file$datapath
  })

  # ---- Update pos after scalefactor mapping ----
  observeEvent(rv$pos, {
    if (!is.null(rv$pos) && !is.null(rv$hires_scale) && is.null(rv$pos$x_hires)) {
      rv$pos <- to_hires_coords(rv$pos, rv$hires_scale)
    }
  })

  expr_vec <- make_expr_vec(
    gene_input = reactive(input$gene),
    counts_rv  = reactive(rv$counts),
    pos_rv     = reactive(rv$pos),
    do_log1p   = reactive(isTRUE(input$log1p))
  )

  rot <- rotateImageServer("rot", image_path_reactive = reactive(rv$img_path))

  sel <- selectSpotsServer(
      "sel",
      pos_reactive  = reactive(rv$pos),      # must include barcode/x_hires/y_hires
      expr_reactive = expr_vec,              # expression vector reactive
      img_reactive  = current_img_obj        # background image (rotated or original)
    )

  # Current background image (prefer rotated if available)
  current_img_obj <- reactive({
    ro <- tryCatch(rot$get(), error = function(e) NULL)
    if (!is.null(ro)) ro else if (!is.null(rv$img)) list(img=rv$img, img_w=rv$img_w, img_h=rv$img_h) else NULL
  })

  # Main plot: HE background + spots; selection inside same plot
  output$he_plotly <- plotly::renderPlotly({
    req(rv$pos, current_img_obj())
    build_he_plotly_figure(
      pos        = rv$pos,
      expr       = expr_vec(),
      img_obj    = current_img_obj(),
      ptsize     = input$ptsize,
      alpha      = input$alpha,
      gene_label = input$gene
    ) %>%
    plotly::layout(dragmode = if (isTruthy(input$sel_mode) && input$sel_mode=="select") "select" else "lasso")
  })

  selected_idx <- reactiveVal(integer(0))
  setup_selection(output, session, selected_idx, reactive(input$sel_mode), reactive(input$clear_sel))

  annotations <- init_annotations()
  setup_annotation_observers(input, output, rv, selected_idx, annotations)
  label_index_list <- reactive(make_label_index_list(rv, annotations))

  multi_distance_df <- setup_multi_distance(output, rv, expr_vec, label_index_list)

  setup_download_selected(
    output         = output,
    id             = "dl_sel",
    pos_r          = reactive(rv$pos),
    expr_vec_r     = expr_vec,
    selected_idx_r = reactive(selected_idx())
  )

  output$dl_multi_distance <- downloadHandler(
    filename = function() paste0("spots_with_signed_distance_multi_",
                                format(Sys.time(), "%Y%m%d_%H%M%S"), ".csv"),
    content = function(file) {
      df <- multi_distance_df()
      # optionally add expression values
      df$expr <- expr_vec()

      # keep only core columns + all dist_* columns
      dist_cols <- grep("^dist_", names(df), value = TRUE)
      base_cols <- intersect(c("barcode","x_hires","y_hires"), names(df))
      keep <- c(base_cols, dist_cols, "expr")
      utils::write.csv(df[, keep, drop = FALSE], file, row.names = FALSE)
    }
  )

  # ---- GAM analysis (multi-structure) ----
  output$dl_gam_results <- downloadHandler(
    filename = function() paste0("gam_results_", format(Sys.time(), "%Y%m%d_%H%M%S"), ".csv"),
    content  = function(file) {
      withProgress(message = "Running GAM fitting ...", value = 0, {
        req(rv$counts, rv$gene_names, rv$pos)
        covar <- as.data.frame(multi_distance_df())   # table with dist_* columns
        stopifnot("barcode" %in% names(covar))

        # offset (normalize controlled by UI)
        off <- compute_log_offset(rv$counts, covar$barcode, normalize = isTRUE(input$log1p))
        covar$size_factor <- off$size_factor
        covar$log_off     <- off$log_off
        counts_aligned    <- off$counts_aligned

        distance_vars <- grep("^dist_", names(covar), value = TRUE)
        if (length(distance_vars) == 0) stop("No dist_* columns found, please generate distances first.")

        # run only top 1000 expressed; apply pre-filter thresholds
        res <- analyze_spatial_genes(
          counts            = counts_aligned,
          genes             = rv$gene_names,
          covar             = covar,
          distance_vars     = distance_vars,
          fam_nb            = mgcv::nb(link = "log"),
          use_interaction   = FALSE,     # disable interactions for speed/stability
          k                 = 6L,
          min_nnz           = 100,       # pre-filter: at least 100 non-zero spots
          min_sum           = 0,
          min_nnz_gene      = 1L,        # per-gene safeguard threshold
          top_n_expressed   = 1000       # only top 1000 genes by total expression
        )

        utils::write.csv(res, file, row.names = FALSE)
      })
    }
  )

  # ---- Read GAM results CSV ----
  gam_df <- reactive({
    req(input$gam_csv)
    df <- tryCatch(
      utils::read.csv(input$gam_csv$datapath, stringsAsFactors = FALSE, check.names = FALSE),
      error = function(e) NULL
    )
    req(!is.null(df))

    # Clean column names (some CSVs may have hidden characters)
    nm <- trimws(names(df))
    nm <- sub("^ï\\.+", "", nm)   # fix BOM prefix
    names(df) <- nm

    if (!"gene" %in% names(df)) stop("CSV missing 'gene' column")
    df
  })

  # ---- Dynamic selection of p_dist_* columns ----
  output$dist_col_ui <- renderUI({
    df <- gam_df()
    pcols <- grep("^p_dist_", names(df), value = TRUE)
    selectInput("dist_col", "Select p-value column for distance", choices = pcols, selected = pcols[1])
  })

  # ---- Pick Top N genes ----
  top_pick <- reactive({
    df <- gam_df(); req(input$dist_col, input$topn_go)
    x <- df[is.finite(as.numeric(df[[input$dist_col]])), , drop = FALSE]
    if (!nrow(x)) stop("This column has no valid p-values")
    x[[input$dist_col]][x[[input$dist_col]] <= 0] <- .Machine$double.xmin
    x$FDR <- p.adjust(as.numeric(x[[input$dist_col]]), method = "BH")
    x <- x[order(x[[input$dist_col]], x$FDR), , drop = FALSE]
    keep_cols <- intersect(c("gene","status","dev_expl", input$dist_col, "FDR"), names(x))
    list(top_df = head(x[, keep_cols, drop = FALSE], input$topn_go),
         universe_symbols = x$gene)
  })

  output$tbl_top_genes <- renderTable({
    tp <- top_pick()
    tp$top_df
  }, striped = TRUE, bordered = TRUE, hover = TRUE)

  # ---- GO enrichment ----
  go_res <- eventReactive(input$run_go, {
    tp <- top_pick(); req(input$go_org)
    run_go_enrichment(
      top_symbols      = tp$top_df$gene,
      universe_symbols = tp$universe_symbols,
      organism         = input$go_org
    )
  })

  output$tbl_go <- renderTable({
    eg <- go_res()
    out <- as.data.frame(eg)
    keep <- intersect(c("ID","Description","GeneRatio","BgRatio","pvalue","p.adjust","qvalue","Count"), names(out))
    out[, keep, drop = FALSE]
  }, striped = TRUE, bordered = TRUE, hover = TRUE)

  output$plot_go <- renderPlot({
    eg <- go_res()
    enrichplot::dotplot(eg, showCategory = min(15, nrow(as.data.frame(eg))))
  })

  output$dl_go_table <- downloadHandler(
    filename = function() paste0("go_enrich_", input$dist_col, "_top", input$topn_go, "_",
                                format(Sys.time(), "%Y%m%d_%H%M%S"), ".csv"),
    content = function(file) {
      eg <- go_res()
      utils::write.csv(as.data.frame(eg), file, row.names = FALSE)
    }
  )

  # ---- Utility: safe gene label ----
  safe_gene_label <- function(x, alt = "expr") {
    if (is.null(x) || length(x) == 0) return(alt)
    x <- as.character(x)[1]
    if (!nzchar(x)) alt else x
  }

  # ---- Read distance file ----
  dist_df <- reactive({
    req(input$dist_csv)
    df <- tryCatch(read.csv(input$dist_csv$datapath, stringsAsFactors = FALSE, check.names = FALSE),
                  error = function(e) NULL)
    req(!is.null(df))
    nm <- trimws(names(df)); nm <- sub("^ï\\.+", "", nm); names(df) <- nm
    if (!("barcode" %in% names(df))) stop("Distance file missing 'barcode' column")
    df
  })

  output$traj_dist_col_ui <- renderUI({
    df <- dist_df()
    dcols <- grep("^dist_", names(df), value = TRUE)
    selectInput("traj_dist_col", "Select distance column", choices = dcols, selected = dcols[1])
  })

  traj_merged <- reactive({
    df <- dist_df(); req(input$traj_dist_col)
    expr_cnt <- NULL
    if (!is.null(rv$counts) && !is.null(rv$pos) && !is.null(input$gene)) {
      bc  <- df$barcode
      ord <- match(bc, colnames(rv$counts))
      if (!anyNA(ord)) {
        expr_cnt <- make_expr_from_counts(
          gene     = input$gene,
          counts   = rv$counts[, ord, drop = FALSE],
          barcodes = bc,
          log1p    = isTRUE(input$log1p)
        )
      }
    }
    merge_dist_expr(df, expr_from_counts = expr_cnt)
  })

  # ---- Plot trajectory + binning ----
  output$traj_plot <- renderPlot({
    df <- traj_merged(); req(input$traj_dist_col)
    dvar <- input$traj_dist_col
    ok <- is.finite(df[[dvar]]) & is.finite(df$expr)
    dx <- df[[dvar]][ok]; dy <- df$expr[ok]

    # Bin summary
    tbl <- bin_summary(dx, dy, bins = input$traj_bins)
    assign("..traj_table_cache", tbl, envir = .GlobalEnv)

    library(ggplot2)
    gene_lab <- safe_gene_label(input$gene, "expr")
    ylab_txt <- if (isTRUE(input$log1p)) paste0("log1p(", gene_lab, ")") else gene_lab

    g <- ggplot(data.frame(x = dx, y = dy), aes(x, y))
    if (isTRUE(input$traj_show_points)) g <- g + geom_point(alpha = 0.25, size = 0.8)
    g <- g +
      geom_smooth(method = "loess", formula = y ~ x, se = TRUE, span = input$traj_span) +
      geom_vline(xintercept = 0, linetype = "dashed", color = "grey50") +
      labs(x = dvar, y = ylab_txt, title = paste("Expression vs", dvar)) +
      theme_minimal(base_size = 12)
    print(g)
  })

  output$traj_table <- renderTable({
    if (exists("..traj_table_cache", envir = .GlobalEnv)) {
      get("..traj_table_cache", envir = .GlobalEnv)
    } else data.frame()
  }, striped = TRUE, bordered = TRUE, hover = TRUE)

  # ---- Download trajectory PNG ----
  output$dl_traj_plot <- downloadHandler(
    filename = function() {
      gene_lab <- safe_gene_label(input$gene, "expr")
      paste0("traj_", input$traj_dist_col, "_", gene_lab, "_",
            format(Sys.time(), "%Y%m%d_%H%M%S"), ".png")
    },
    content = function(file) {
      df <- traj_merged(); req(input$traj_dist_col)
      dvar <- input$traj_dist_col
      ok <- is.finite(df[[dvar]]) & is.finite(df$expr)

      library(ggplot2)
      gene_lab <- safe_gene_label(input$gene, "expr")
      ylab_txt <- if (isTRUE(input$log1p)) paste0("log1p(", gene_lab, ")") else gene_lab

      png(file, width = 1600, height = 1000, res = 150)
      g <- ggplot(data.frame(x = df[[dvar]][ok], y = df$expr[ok]), aes(x, y))
      if (isTRUE(input$traj_show_points)) g <- g + geom_point(alpha = 0.25, size = 0.8)
      g <- g +
        geom_smooth(method = "loess", formula = y ~ x, se = TRUE, span = input$traj_span) +
        geom_vline(xintercept = 0, linetype = "dashed", color = "grey50") +
        labs(x = dvar, y = ylab_txt, title = paste("Expression vs", dvar)) +
        theme_minimal(base_size = 14)
      print(g)
      dev.off()
    }
  )

  # ---- Download trajectory binning table ----
  output$dl_traj_table <- downloadHandler(
    filename = function() paste0("traj_bins_", input$traj_dist_col, "_",
                                format(Sys.time(), "%Y%m%d_%H%M%S"), ".csv"),
    content = function(file) {
      tbl <- if (exists("..traj_table_cache", envir = .GlobalEnv)) {
        get("..traj_table_cache", envir = .GlobalEnv)
      } else data.frame()
      utils::write.csv(tbl, file, row.names = FALSE)
    }
  )

  # ---- Debug info (optional) ----
  output$dbg <- renderPrint({
    list(
      have_counts = !is.null(rv$counts),
      have_pos    = !is.null(rv$pos),
      have_scale  = !is.null(rv$hires_scale),
      have_img    = !is.null(rv$img),
      img_class   = if (!is.null(rv$img)) class(rv$img)
    )
  })
}



In [4]:
shinyApp(ui, server)


Listening on http://127.0.0.1:6118

Loading required namespace: magick

Warning message:
“The 'plotly_selected' event tied a source ID of '' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_selected')` to the plot (`p`) that you wish to obtain event data from.”
Warning message:
“The 'plotly_selected' event tied a source ID of 'main' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_selected')` to the plot (`p`) that you wish to obtain event data from.”
Warning message:
“The 'plotly_selected' event tied a source ID of 'main' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_selected')` to the plot (`p`) that you wish to obtain event data from.”
